## 필요한 패키지 불러오기

In [ ]:
import requests

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
import pymysql

In [ ]:
from selenium import webdriver

In [ ]:
from selenium.webdriver.common.keys import Keys

In [ ]:
import time as time

In [ ]:
import getpass

In [ ]:
import urllib.request

In [ ]:
import random

In [ ]:
from time import sleep

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

## 데이터 크롤링 시작하기

In [ ]:
driver = webdriver.Chrome('크롬드라이버가 있는 경로 넣기') # Chromedriver PATH

In [ ]:
driver.get("https://www.instagram.com/accounts/login/")

In [ ]:
# driver.maximize_window() # 인터넷 창아, 화면을 좀 전체화며으로 키워 줘! 라는 명령어

### 인스타 자동 로그인하기

In [ ]:
username = getpass.getpass("Input ID : ") # getpass 는 모든 정보를 가려서 보여주는 장점

In [ ]:
password = getpass.getpass("Input PWD : ")

In [ ]:
element_id = driver.find_element_by_name("username")

In [ ]:
element_id.send_keys(username)

In [ ]:
element_password = driver.find_element_by_name("password")

In [ ]:
element_password.send_keys(password)

In [ ]:
sleep(5)

In [ ]:
driver.find_element_by_css_selector('.sqdOP.L3NKy.y3zKF').click()

## 해쉬태그로 검색하기

In [ ]:
#driver.get('https://www.instagram.com/explore/tags/국내여행/') #www.instagram.com/explore/tags/검색하고자 하는 패턴/

In [ ]:
driver.get('https://www.instagram.com/explore/tags/국내여행/')

In [ ]:
# 데이터를 저장할 dictionary
insta_dict = {'id':[],
              'date':[],
              'like':[],
              'text':[],
              'hashtag':[]}

In [ ]:
# 첫 번째 게시물 클릭
first_post = driver.find_element_by_class_name('eLAPa')
first_post.click()

### 자동으로 옆으로 넘기면서 데이터 수집하기

In [ ]:
seq = 0
start = time.time()

In [ ]:
while True:
    try:
        if driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow'):
            if seq % 20 == 0:
                print('{}번째 수집 중'.format(seq), time.time() - start, sep = '\t')
 
            ## id 정보 수집
            try:
                info_id = driver.find_element_by_css_selector('h2._6lAjh').text
                insta_dict['id'].append(info_id)
            except:
                info_id = driver.find_element_by_css_selector('div.C4VMK').text.split()[0]
                insta_dict['id'].append(info_id)
 
 
            ## 시간정보 수집 
            time_raw = driver.find_element_by_css_selector('time.FH9sR.Nzb55')
            time_info = pd.to_datetime(time_raw.get_attribute('datetime')).normalize()
            insta_dict['date'].append(time_info)
 
            ## like 정보 수집
            try:
                driver.find_element_by_css_selector('button.sqdOP.yWX7d._8A5w5')
                like = driver.find_element_by_css_selector('button.sqdOP.yWX7d._8A5w5').text
                insta_dict['like'].append(like)
 
            except:
                insta_dict['like'].append('영상')
 
            ##text 정보수집
            raw_info = driver.find_element_by_css_selector('div.C4VMK').text.split()
            text = []
            for i in range(len(raw_info)):
                ## 첫번째 text는 아이디니까 제외 
                if i == 0:
                    pass
                ## 두번째부터 시작 
                else:
                    if '#' in raw_info[i]:
                        pass
                    else:
                        text.append(raw_info[i])
            clean_text = ' '.join(text)
            insta_dict['text'].append(clean_text)
 
            ##hashtag 수집
            raw_tags = driver.find_elements_by_css_selector('a.xil3i')
            hash_tag = []
            for i in range(len(raw_tags)):
                if raw_tags[i].text == '':
                    pass
                else:
                    hash_tag.append(raw_tags[i].text)
 
            insta_dict['hashtag'].append(hash_tag)
 
            seq += 1
 
            if seq == 500:
                break
 
            driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
            time.sleep(4) # sleep 4초 간격으로, 너무 빠르시 컴퓨터 과부하
            
        else:
            break
            
    except:
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(4)

In [ ]:
test = pd.DataFrame.from_dict(insta_dict)

In [ ]:
test.head()

In [ ]:
print(test)

In [ ]:
test.to_csv('국내여행db20_0713.csv', encoding='utf-8-sig') #csv로 저장